In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pathlib
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

print(tf.__version__)

In [ ]:
data_dir='/Users/daoud/Desktop/HGCAL/HGCROC/HGCROC_image_dataset_Augmented'
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)

In [ ]:
acceptable = list(data_dir.glob('Acceptable/*'))
PIL.Image.open(str(acceptable[0]))
print (len(acceptable))

In [ ]:
empty = list(data_dir.glob('Empty/*'))
PIL.Image.open(str(empty[0]))
print (len(empty))

In [ ]:
flipped = list(data_dir.glob('Flipped/*'))
PIL.Image.open(str(flipped[0]))

In [ ]:
mispositionned = list(data_dir.glob('Mispositionned/*'))
PIL.Image.open(str(mispositionned[0]))

In [ ]:
one_on_top_of_another = list(data_dir.glob('One_on_top_of_another/*'))
PIL.Image.open(str(one_on_top_of_another[0]))
print (len(one_on_top_of_another))

In [ ]:
batch_size = 32
img_height = 100
img_width = 180


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
class_names = train_ds.class_names
print(class_names)


In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
normalization_layer = layers.Rescaling(1./255)


In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
epochs=50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
def cropped_image(image):
    #print("-> Cropped gray image")
    img_crop = image[350:590,260:730]
    return img_crop

In [ ]:
from PIL import Image

Validation_path = '/Users/daoud/Desktop/HGCAL/HGCROC/HGCROC images - all/Validation Pictures'

plt.figure(figsize = (10,10))


for image in os.listdir(Validation_path):


    print("\n\n\n=>Image initial name : ",image)
    


    #-----------------------------------------
    img = tf.keras.utils.load_img(Validation_path+"/" + image )
    
    #print(img.size)
    
    
    #Image cropped
    left=280
    right=750
    top=350
    bottom=590
    img=img.crop((left, top, right, bottom)) 
    
    size = 180, 100

    img=img.resize(size)
    #print(img.size)
    
    # Display cropped image
    plt.imshow(img)
    plt.axis('off')
    plt.show() 
    
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    print("SCORE : ",score)
    print("Predicitons : ",predictions)
    
    print(
        "\nThis image most likely belongs to {} with a {:.2f} percent confidence. "
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

    print(
        "Cette photo appartient à la classe {} avec une confiance de {:.2f} % "
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )